### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import oci
import os
import base64
import ocifs
import PIL.Image as Image
import io
import re

#Inicialização dos clients
config = oci.config.from_file('config')
ai_vision_client = oci.ai_vision.AIServiceVisionClient(config=config)
fs = ocifs.OCIFileSystem()
object_storage_client = oci.object_storage.ObjectStorageClient(config)

#Lendo as imagens do bucket. Esta função fs.glob possui a seguinte sintaxe:
#"oci://<bucket_name>@<namespace>/arquivos"

img_list = fs.glob("oci://images_comprovantes_pagamento@id3kyspkytmr/*.jpg")

In [22]:
#Essas funções servem para manter o tracking dos nomes de cada documento, caso eles utilizem algum padrão de nomenclatura para
#podermos saber qual texto refere-se a qual documento

#Neste script em particular, não são utilizadas

name_list = []
for i, item in enumerate(img_list):
    name = item.rsplit('/', 1)[1]
    name_list.append(name)
    
code_list = []
for i in range(0, len(txt_list)):
    codigo = txt_list[i]
    s = [float(s) for s in re.findall(r'-?\d+\.?\d*', codigo)]
    code_list.append(int(s[1]))

In [ ]:
#Faço um loop para cara imagem lida do bucket para extração de textos

for i, file in enumerate(img_list):
        with fs.open(file) as f:
            content = f.read()
            
            encoded_string = base64.b64encode(content, altchars=None)
            decoded_string = encoded_string.decode("utf-8", "ignore")

                
            #Não estranhar o idioma marcado como inglês, a ferramenta também lida com textos em pt-br, mas há um hardblock interno
            #que bloqueia a extração de texto se o idioma detctado for não-inglês, então travamos ele na chamada da API.
            
            analyze_document_response = ai_vision_client.analyze_document(
                    analyze_document_details=oci.ai_vision.models.AnalyzeDocumentDetails(
                    compartment_id = "ocid1.compartment.oc1..aaaaaaaal63rmctoojg7q2pvdpeuqknebyaqg3h7gcci6whf74ht7tfapl4q",
                    features=[
                        oci.ai_vision.models.DocumentTextDetectionFeature(
                            feature_type="TEXT_DETECTION")],
                    language='ENG', #Sim, o idioma é ENG mesmo. A ferramenta tem um hardblock para textos não-inglês
                    document=oci.ai_vision.models.InlineDocumentDetails(
                    source="INLINE",
                    data=decoded_string)))

            #Extraimos as palavras da response da API e limpaos o texto, pois vem recheado de colchetes e aspas
            
            words = str([word.text for page in analyze_document_response.data.pages for word in page.words])
            name = f"{i}.txt" 
            
            words_str_1 = words.replace("',",'')
            words_str_2 = words_str_1.replace("'",'')
            words_str_3 = words_str_2.replace("[",'')
            words_str_4 = words_str_3.replace("]",'')

            #Colocamos o resultado no bucket correspondente que criamos para receber estes textos
            
            put_object_response = object_storage_client.put_object(
                namespace_name="id3kyspkytmr", #TO DO
                bucket_name="images_comprovantes_pagamento_formatado", #TO DO
                object_name=name,
                put_object_body = words_str_4,
                content_type="text/plain")
            
            print("Documento", name, "upado!")